**PyTorch入門 4. モデル構築**  
https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_4_buildmodel_tutorial_js.ipynb


In [1]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3054, -0.7628, -0.0787, -0.1500,  0.0687,  0.0381, -0.2981,  0.1683,
         -0.2127, -0.0276, -0.1596, -0.5292, -0.0328,  0.2545,  0.2304,  0.1731,
          0.0498, -0.3459, -0.0726, -0.6210],
        [ 0.6497, -0.2855, -0.1859, -0.1730,  0.3562,  0.4317, -0.0844,  0.5825,
         -0.3875, -0.0501, -0.5301, -0.2985, -0.0074,  0.0474, -0.1206,  0.0419,
         -0.0383, -0.2056, -0.2336, -0.4486],
        [ 0.3816, -0.8751,  0.2281, -0.2968, -0.1853,  0.1831,  0.2187,  0.4348,
         -0.0806,  0.1766,  0.0326, -0.1978,  0.0988,  0.1435,  0.1129, -0.1512,
         -0.1709, -0.2121, -0.0447, -0.4519]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3054, 0.0000, 0.0000, 0.0000, 0.0687, 0.0381, 0.0000, 0.1683, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2545, 0.2304, 0.1731, 0.0498, 0.0000,
         0.0000, 0.0000],
        [0.6497, 0.0000, 0.0000, 0.0000, 0.3562, 0.4317, 0.0000, 0.5825, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0474, 0.00

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0211, -0.0243,  0.0198,  ...,  0.0135, -0.0166, -0.0211],
        [ 0.0021,  0.0092, -0.0007,  ...,  0.0265,  0.0081, -0.0171]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0314, 0.0081], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0323, -0.0046,  0.0142,  ..., -0.0222,  0.0373, -0.0041],
        [ 0.0111,  0.0382, -0.0346,  ..., -0.0042,  0.0096, -0.0040]],
       device='cuda:0'